In [11]:
import cobra

modelo = cobra.io.read_sbml_model('AraGEM_Photo_after_Pipeline.xml')

In [12]:
modelo.objective_direction = "max"

In [13]:
modelo.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


In [6]:
modelo.reactions.get_by_id("Biomass_Total").upper_bound = 0.11
modelo.reactions.get_by_id("Biomass_Total").lower_bound = 0.11
modelo.objective = "Ex16_Day"
modelo.objective_direction = "max"

In [12]:
modelo.reactions.get_by_id("Biomass_Total").upper_bound = 1000
modelo.reactions.get_by_id("Biomass_Total").lower_bound = 0
modelo.objective = "Ex16_Day"
modelo.objective_direction = "max"

In [19]:
modelo.reactions.get_by_id("Biomass_Total").upper_bound = 0.11
modelo.reactions.get_by_id("Biomass_Total").lower_bound = 0.11
modelo.objective = "Biomass_Total"
modelo.objective_direction = "max"

In [26]:
modelo.reactions.get_by_id("Biomass_Total").upper_bound = 1000
modelo.reactions.get_by_id("Biomass_Total").lower_bound = 0
modelo.objective = "Biomass_Total"
modelo.objective_direction = "max"

In [7]:
from cobra.flux_analysis import pfba

solution = pfba(modelo)
R02732_c = solution.fluxes["R02732_c_Day"] + solution.fluxes["R02732_c_Night"]
R01070N_c = solution.fluxes["R01070N_c_Day"] + solution.fluxes["R01070N_c_Night"]
R00762_c = solution.fluxes["R00762_c_Day"] + solution.fluxes["R00762_c_Night"]

chex2 = abs(round(((R02732_c - R01070N_c) + R00762_c) / 2, 2))
print(chex2)

Infeasible: None (infeasible).

In [28]:
R04779_p = solution.fluxes["R04779_p_Day"] + solution.fluxes["R04779_p_Night"]
R00762_p = solution.fluxes["R00762_p_Day"] + solution.fluxes["R00762_p_Night"]
R01070N_p = solution.fluxes["R01070N_p_Day"] + solution.fluxes["R01070N_p_Night"]


phex1 = abs(round(((R04779_p - R00762_p) - R01070N_p) / 2, 2))
phex1

0.0

In [29]:
R01061_c = solution.fluxes["R01061_c_Day"] + solution.fluxes["R01061_c_Night"]
R01061_p = solution.fluxes["R01061_p_Day"] + solution.fluxes["R01061_p_Night"]
R01063_p = solution.fluxes["R01063_p_Day"] + solution.fluxes["R01063_p_Night"]
R01512_c = solution.fluxes["R01512_c_Day"] + solution.fluxes["R01512_c_Night"]
R01518_c = solution.fluxes["R01518_c_Day"] + solution.fluxes["R01518_c_Night"]
R01518_p = solution.fluxes["R01518_p_Day"] + solution.fluxes["R01518_p_Night"]
R00658_c = solution.fluxes["R00658_c_Day"] + solution.fluxes["R00658_c_Night"]
R00658_p = solution.fluxes["R00658_p_Day"] + solution.fluxes["R00658_p_Night"]
R01512_p = solution.fluxes["R01512_p_Day"] + solution.fluxes["R01512_p_Night"]

Cphex = abs(round(((R01061_c + R01061_p - R01063_p - R01512_c - R01512_p)/2 + (R01518_c + R01518_p) + (R00658_c + R00658_p))/3, 2))
Cphex

21.49

In [30]:
R00200_c = solution.fluxes["R00200_c_Day"] + solution.fluxes["R00200_c_Night"]

chex3 = abs(round(R00200_c,2))
chex3

36.19

In [31]:
R00200_p = solution.fluxes["R00200_p_Day"] + solution.fluxes["R00200_p_Night"]
R00206_p = solution.fluxes["R00206_p_Day"] + solution.fluxes["R00206_p_Night"]

phex2 = abs(round(R00200_p - R00206_p, 4))
phex2

8.5548

In [32]:
R00351_m = solution.fluxes["R00351_m_Day"] + solution.fluxes["R00351_m_Night"]
R00209_m = solution.fluxes["R00209_m_Day"] + solution.fluxes["R00209_m_Night"]
TCM1 = solution.fluxes["TCM1_Day"] + solution.fluxes["TCM1_Night"]

tca2 = abs(round((-TCM1 + R00351_m + R00209_m)/3, 2))
tca2

0.0

In [33]:
import pandas as pd

dados = {'Valores Artigo (pFBA)': ['11.38', '1.719', '19.07', '0.0065', '5.40', '9.51'],
         'Valores Artigo (MFA)' : ['7.33', '2.89', '23.96', '11.18', '7.95', '10.27'],
         'OF: max of Ex16_Day (pFBA)': ['9.27', '39.42', '0.33', '0.36', '0.0229', '0.01'],
         'OF: max of Ex16_Day (pFBA)*': ['9.84', '40.54', '0.17', '0.19', '0.0029', '0.0'],
         'OF: max of Biomass_Total (pFBA)' : ['0.02', '0.0', '0.13', '0.3', '0.0557', '0.0'],
         'OF: max of Biomass_Total (pFBA)*' : ['8.81', '0.0', '21.49', '36.19', '8.5548', '0.0']}

df = pd.DataFrame(dados)

nomes = ['chex2', 'phex1', 'Cphex', 'chex3', 'phex2', 'tca2']
df.index = nomes

df

,Valores Artigo (pFBA),Valores Artigo (MFA),OF: max of Ex16_Day (pFBA),OF: max of Ex16_Day (pFBA)*,OF: max of Biomass_Total (pFBA),OF: max of Biomass_Total (pFBA)*
chex2,11.38,7.33,9.27,9.84,0.02,8.81
phex1,1.719,2.89,39.42,40.54,0.0,0.0
Cphex,19.07,23.96,0.33,0.17,0.13,21.49
chex3,0.0065,11.18,0.36,0.19,0.3,36.19
phex2,5.40,7.95,0.0229,0.0029,0.0557,8.5548
tca2,9.51,10.27,0.01,0.0,0.0,0.0


*As constraints para a reação de Biomass_Total estão a (0, 1000) nestas hipóteses